In [ ]:
import numpy as np
import pandas as pd
import requests
import re
import json
import datetime as dt 
from datetime import datetime, timedelta, date

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all

import xlsxwriter as xl

In [ ]:
# Start included, end excluded.

#update 'start' date in this cell and also the file name in cell 70 & 71 (out of 73 cells)

last_month = dt.date.today().replace(day=1) - dt.timedelta(days=1)
start= dt.datetime.strptime("2020-01-01", "%Y-%m-%d")
end=start.replace(year = start.year + 1)

query_vars = dict(
    snapshot = last_month.strftime("%Y-%m"),
    start= start.strftime('%Y-%m-%d'), 
    end = end.strftime('%Y-%m-%d'),
    pv_start = start.strftime("%Y%m"),
    pv_end = end.strftime("%Y%m"),
    
    ner_start = start.strftime("%Y-%m"),
    ner_end = end.strftime("%Y-%m"),
)

# List of wikis

In [ ]:
# Gather all content wikis
wikis = wmf.hive.run("""
SELECT
  database_code,
  database_group AS project_code,
  language_code,
  CONCAT("https://", domain_name) AS domain_name,
  language_name,
  english_name as wiki_name
FROM canonical_data.wikis
WHERE
  database_group in (
    "commons", "incubator", "foundation", "mediawiki", "meta", "sources",
    "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
    "wikisource", "wikiversity", "wikivoyage", "wiktionary"
  ) AND
  status = "open" AND
  visibility = "public" AND
  editability = "public"
""")

In [ ]:
wikis.sample(10)

# Data

In [ ]:
def merge_in(df, on="database_code"):
    global wikis
    wikis = pd.merge(wikis, df, how="left", on=on).fillna(0)
    
def top_10(df, col):
    return df.sort_values(col, ascending=False).head(10)
  
def rename_df(df):
    return df.rename({"wiki": "database_code", "domain": "domain_name"}, axis=1)

In [ ]:
wikis.info()

## Monthly active editors

In [ ]:
mae = wmf.hive.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_active_editors,
    SUM(
        CAST(TRUNC(user_registration, 'MM') = TRUNC(month, 'MM') AS INT)
        )/ 12 AS monthly_new_active_editors
FROM cchen.editor_month
WHERE
    content_edits >= 5 
    AND month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    --AND user_id IS NOT NULL
    AND bot_by_group = FALSE 
    AND (
        user_name not regexp "bot\\b" or
        user_name in ("Paucabot", "Niabot", "Marbot")    
    )    
GROUP BY wiki
""".format(**query_vars))

In [ ]:
mae

In [ ]:
merge_in(mae)

## Monthly unique devices

In [ ]:
mud = wmf.hive.run("""
SELECT
    regexp_replace(
        regexp_replace(
            regexp_replace(domain, "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.') AS domain_name,
  SUM(uniques_estimate) / 12 AS monthly_unique_devices
FROM wmf.unique_devices_per_domain_monthly
WHERE 
    CONCAT(year,LPAD(month,2,'0')) >= "{pv_start}" and
    CONCAT(year,LPAD(month,2,'0')) < "{pv_end}"
GROUP BY    
    regexp_replace(
        regexp_replace(
            regexp_replace(domain, "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.')
""".format(**query_vars))

mud["domain_name"] = "https://" + mud["domain_name"]

top_10(mud, "monthly_unique_devices")

In [ ]:
merge_in(mud, on="domain_name")

## Overall SIZE rank

In [ ]:
SIZE = np.sqrt(wikis["monthly_unique_devices"] * wikis["monthly_active_editors"])
rank = SIZE.rank(method="min", na_option="bottom", ascending=False)
wikis["overall_SIZE_rank"] = rank

## New editor retention

In [ ]:
with open("queries/new_editor_retention.hql") as f:
    q = f.read()

ner = wmf.hive.run(
    q.format(start = "{ner_start}", end = "{ner_end}").format(**query_vars))

In [ ]:
top_10(ner, "new_editor_retention")

In [ ]:
merge_in(ner)

## Mobile editing proportion

In [ ]:
mep = wmf.hive.run(
"""
SELECT 
    wiki AS database_code,
    SUM(mobile_web_edits + mobile_app_edits) / SUM(edits) AS mobile_editing_proportion
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}"
    -- A user is a bot if they have a matching name or have the bot flag on *any* wiki
    -- See https://meta.wikimedia.org/wiki/Research:Active_editor and https://meta.wikimedia.org/wiki/Research:Bot_user
    AND user_id != 0
    AND NOT bot_by_group 
    AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

In [ ]:
top_10(mep, "mobile_editing_proportion")

In [ ]:
merge_in(mep)

## Bot editing proportion

In [ ]:
bep = wmf.hive.run("""
SELECT
   database_code,
   SUM(CASE WHEN user_is_bot THEN edit_count END)/ SUM(edit_count) AS bot_editing_proportion
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts  >= "{start}" 
   AND ts  < "{end}"
   AND snapshot = "{snapshot}"
GROUP BY database_code
""".format(**query_vars))

top_10(bep, "bot_editing_proportion")

In [ ]:
merge_in(bep)

## Anonymous editing proportion

In [ ]:
aep = wmf.hive.run("""
SELECT
   database_code,
   SUM(IF(user_is_anonymous, edit_count, 0)) / SUM(edit_count)  AS anonymous_editing_proportion
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts >= "{start}" 
   AND ts < "{end}"
   AND NOT user_is_bot
   AND snapshot = "{snapshot}"
GROUP BY database_code
""".format(**query_vars))

top_10(aep, "anonymous_editing_proportion")

In [ ]:
merge_in(aep)

## Majority-mobile editors proportion

In [ ]:
mmep = wmf.hive.run("""
SELECT 
   wiki AS database_code,
   SUM(CASE WHEN mobile_editing_proportion > 0.5 THEN 1 END) / COUNT(*) as majority_mobile_editors_proportion
FROM 
(
    SELECT 
        wiki,
        SUM(mobile_web_edits + mobile_app_edits) / SUM(edits) AS mobile_editing_proportion
    FROM cchen.editor_month
    WHERE
        month >= "{start}" 
        AND month < "{end}" 
        AND user_id != 0
        AND NOT bot_by_group 
        AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
    GROUP BY wiki, user_name
) user_edits
GROUP BY wiki
""".format(**query_vars))
top_10(mmep, "majority_mobile_editors_proportion")

In [ ]:
merge_in(mmep)

## Revert rate

In [ ]:
rr = wmf.hive.run("""
    SELECT
        wiki_db AS database_code,
        SUM(IF(revision_is_identity_reverted, 1, 0)) / COUNT(*) AS revert_rate
        --SUM(CAST(revision_is_identity_reverted AS INT)) / COUNT(*) AS revert_rate
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}" 
        AND SIZE(event_user_is_bot_by_historical) = 0 
    GROUP BY wiki_db
""".format(**query_vars))

top_10(rr, "revert_rate")

In [ ]:
merge_in(rr)

## Monthly pageviews

In [ ]:
pageviews = hive.run("""
SELECT CONCAT("https://", project, ".org") AS domain_name, 
       SUM(view_count)/12 AS monthly_average_pageviews
FROM wmf.projectview_hourly
WHERE
    agent_type = "user" 
    AND CONCAT(year, LPAD(month, 2, '0')) >= "{pv_start}" 
    AND CONCAT(year, LPAD(month, 2, '0')) < "{pv_end}"
GROUP BY CONCAT("https://", project, ".org")
""".format(**query_vars))
top_10(pageviews, "monthly_average_pageviews")

In [ ]:
pageviews = pageviews.replace("https://wikidata.org", "https://www.wikidata.org")

In [ ]:
merge_in(pageviews, on="domain_name")

## Mobile pageviews proportion

In [ ]:
mpp = wmf.hive.run("""
SELECT 
    CONCAT("https://", project, ".org") AS domain_name,
    SUM(if(access_method = "mobile web", view_COUNT, 0)) / SUM(view_COUNT) AS mobile_web_pageviews_proportion,
    SUM(if(access_method = "mobile app", view_COUNT, 0)) / SUM(view_COUNT) AS mobile_app_pageviews_proportion
FROM wmf.projectview_hourly
WHERE
    agent_type = "user" 
    AND CONCAT(year,LPAD(month,2,'0')) >= "{pv_start}" 
    AND CONCAT(year,LPAD(month,2,'0')) < "{pv_end}" 
GROUP BY CONCAT("https://", project, ".org")
""".format(**query_vars))

In [ ]:
mpp = mpp.replace("https://wikidata.org", "https://www.wikidata.org")

In [ ]:
top_10(mpp, "mobile_web_pageviews_proportion")

In [ ]:
merge_in(mpp, on="domain_name")

## Monthly active administrators

In [ ]:
maa = hive.run("""
SELECT
    wiki as database_code,
    sum(monthly_active_administrators) / 12 as monthly_active_administrators
FROM (
    SELECT
        wiki_db as wiki,
        substr(log_timestamp, 1, 6) as month,
        count(distinct log_actor) as monthly_active_administrators
    from wmf_raw.mediawiki_logging
    WHERE
        log_type in ("block", "delete", "protect", "rights")
        -- Omit the "delete_redir", "move_prot", and "autopromote" actions, which can be done by regular users
        AND log_action not in ("autopromote", "delete_redir", "move_prot")
        AND log_timestamp >= "{pv_start}" 
        AND log_timestamp < "{pv_end}" 
        AND snapshot = "{snapshot}"
    GROUP BY wiki_db, substr(log_timestamp, 1, 6)
) mae
GROUP BY wiki
""".format(**query_vars))

top_10(maa, "monthly_active_administrators")

In [ ]:
merge_in(maa)

## Monthly non-bot edits

In [ ]:
mnbe = wmf.hive.run("""
SELECT
   database_code,
   SUM(edit_count) /12 AS monthly_nonbot_edits
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts >= "{start}" 
   AND ts < "{end}" 
   AND NOT user_is_bot
   AND snapshot = "{snapshot}" 
GROUP BY database_code
""".format(**query_vars))

top_10(mnbe, "monthly_nonbot_edits")

In [ ]:
merge_in(mnbe)

## Edits Gini coefficient

In [ ]:
user_edits = wmf.hive.run("""
    SELECT
        wiki_db AS wiki,
        COUNT(*) AS user_edits
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}" 
        AND SIZE(event_user_is_bot_by_historical) = 0
        --event_user_is_bot_by_name = false 
        --array_contains(event_user_groups, "bot") = false
        --array_contains(event_user_is_bot_by, "NULL")= false
    GROUP BY event_user_id, wiki_db
""".format(**query_vars))

In [ ]:
# FROM https://github.com/oliviaguest/gini
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # FROM:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array = array + 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

In [ ]:
egc = user_edits.groupby("wiki").apply(lambda g: gini(g["user_edits"].values)).reset_index()

In [ ]:
egc.columns = ["database_code", "edits_Gini_coefficient"]

In [ ]:
merge_in(egc)

## Monthly editors

In [ ]:
me = wmf.hive.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_editors
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    AND bot_by_group = FALSE
    AND (user_name not regexp "bot\\b" or user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

top_10(me, "monthly_editors")

In [ ]:
merge_in(me)

## Unique devices per editor

In [ ]:
wikis["unique_devices_per_editor"] = wikis["monthly_unique_devices"] / wikis["monthly_editors"]

In [ ]:
wikis = wikis.replace([np.inf], 0)

## Article COUNT

In [ ]:
#wikis_list = wikis["wiki"].tolist()
wikis_list = wikis["database_code"].tolist()

In [ ]:
#as of 09/19
wikis_list_not_working = ['alswiktionary', 'alswikibooks', 'alswikiquote', 'mowiki', 'mowiktionary']

In [ ]:
wikis_list_clean = [x for x in wikis_list if x not in wikis_list_not_working]

In [ ]:
ac = wmf.mariadb.run("""
SELECT
    database() AS database_code,
    ss_good_articles AS article_COUNT
FROM site_stats
""", wikis_list_clean)

In [ ]:
top_10(ac, "article_COUNT")

In [ ]:
merge_in(ac)

## Cumulative content edits

In [ ]:
cce = wmf.hive.run("""
    SELECT
        wiki_db AS database_code,
        COUNT(*) AS cumulative_content_edits
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND page_namespace_is_content = true 
        AND SIZE(event_user_is_bot_by_historical) = 0 
        AND array_contains(event_user_groups, "bot") = false
    GROUP BY wiki_db
""".format(**query_vars))

In [ ]:
top_10(cce, "cumulative_content_edits")

In [ ]:
merge_in(cce)

## Edits per content page

In [ ]:
wikis["edits_per_content_page"] = wikis["cumulative_content_edits"] / wikis["article_COUNT"]

## Script direction

In [ ]:
rtl_url = "https://noc.wikimedia.org/conf/dblists/rtl.dblist"
rtl_wikis = pd.Series(requests.get(rtl_url).text.split("\n"))
rtl = pd.DataFrame({"database_code": rtl_wikis, "script_direction": "right-to-left"})

merge_in(rtl)
wikis["script_direction"] = wikis["script_direction"].replace([0], "left-to-right")

## Monthly structured discussions messages

In [ ]:
msdm = wmf.mariadb.run(
"""
SELECT
    rev_user_wiki AS database_code,
    COUNT(*) / 12 AS monthly_structured_discussions_messages
FROM flowdb.flow_revision
WHERE
    rev_change_type in ("new-post", "reply") 
    AND date_format(FROM_unixtime(
        (conv(substring(hex(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") >= "{start}" 
    AND date_format(FROM_unixtime(
        (conv(substring(hex(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") < "{end}"
GROUP BY rev_user_wiki
""".format(**query_vars), "wikishared")

top_10(msdm, "monthly_structured_discussions_messages")

In [ ]:
merge_in(msdm)

## Visual edits

In [ ]:
ve = wmf.hive.run("""
SELECT 
    wiki AS database_code,
    SUM(visual_edits) / SUM(edits) AS visual_edits
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}"
    AND user_id != 0
    AND NOT bot_by_group 
    AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

top_10(ve, "visual_edits")

In [ ]:
merge_in(ve)

## Mobile unique devices

In [ ]:
mob_ud = wmf.hive.run("""
SELECT
    regexp_replace(
        regexp_replace(
            regexp_replace(regexp_replace(domain, "www\\\\.", ""), "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.') AS domain_name,
    SUM(if((domain regexp '^m\\\\.' or  domain regexp '\\\\.m\\\\.'), uniques_estimate, 0)) AS mobile_COUNT,
    SUM(uniques_estimate) AS total_COUNT,
    SUM(
        IF((domain regexp '^m\\\\.' or  domain regexp '\\\\.m\\\\.'), uniques_estimate, 0)
    ) / SUM(uniques_estimate) AS mobile_unique_devices
FROM wmf.unique_devices_per_domain_monthly
WHERE 
    CONCAT(year, LPAD(month, 2, '0')) >= "{pv_start}" AND
    CONCAT(year, LPAD(month, 2, '0')) < "{pv_end}"  
GROUP BY    
    regexp_replace(
        regexp_replace(
            regexp_replace(regexp_replace(domain, "www\\\\.", ""), "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.')
""".format(**query_vars))

mob_ud["domain_name"] = "https://" + mob_ud["domain_name"]

In [ ]:
mob_ud = mob_ud.replace("https://wikidata.org", "https://www.wikidata.org")

In [ ]:
top_10(mob_ud, "mobile_unique_devices")

In [ ]:
merge_in(mob_ud, on="domain_name")

# Readying for spreadsheet

In [ ]:
wikis.columns.tolist()

In [ ]:
wikis.head()

In [ ]:
wikis = wikis[[
    'overall_SIZE_rank',
    'monthly_unique_devices',
    'mobile_unique_devices',
    'mobile_web_pageviews_proportion',
    'mobile_app_pageviews_proportion',
    'unique_devices_per_editor',
    'monthly_editors',
    'monthly_active_editors',
    'monthly_active_administrators',
    'majority_mobile_editors_proportion',
    'monthly_new_active_editors',
    'new_editor_retention',
    'monthly_nonbot_edits',
    'bot_editing_proportion',
    'mobile_editing_proportion',
    'visual_edits',
    'anonymous_editing_proportion',
    'revert_rate',
    'edits_Gini_coefficient',
    'monthly_structured_discussions_messages',
    'article_COUNT',
    'cumulative_content_edits',
    'edits_per_content_page',
    'script_direction',
    'database_code',
    'project_code',
    'language_code',
    'domain_name',
    'language_name',
    'project_code',
    'wiki_name',
]]

In [ ]:
wikis.rename(columns={
    'article_COUNT':'content_pages',
    'anonymous_editing_proportion': 'anonymous_edits',
    'mobile_editing_proportion': 'mobile_edits',
    'bot_editing_proportion':'bot_edits',
    'new_editor_retention':'second_month_editor_retention',
    'majority_mobile_editors_proportion':'majority_mobile_editors',
    'mobile_app_pageviews_proportion':'mobile_app_pageviews',
    'mobile_web_pageviews_proportion':'mobile_web_pageviews',
    'domain_name':'domain',
    'wiki_name': 'wiki'
}, inplace=True)

#drop the underscore in headers
wikis.columns = wikis.columns.str.replace('_', ' ')

#add url column and put at the front
wikis['url'] = wikis['wiki'] + '#' + wikis['domain']
cols = wikis.columns.tolist()
cols = cols[-1:] + cols[:-1]
wikis = wikis[cols]

#sort by size
wikis = wikis.sort_values("overall SIZE rank").fillna(0).reset_index(drop=True)

#format to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format
#format floats to use comma separators https://pandas.pydata.org/pandas-docs/version/0.23.4/options.html
pd.options.display.float_format = '{:,}'.format

In [ ]:
def make_hyperlink(val):
    wiki, domain = val.split('#')
    #url = "https://custom.url/{}"
    return '=HYPERLINK("{domain}", "{wiki}")'.format(domain=domain, wiki=wiki)

wikis['url'] = wikis['url'].apply(lambda x: make_hyperlink(x))

In [ ]:
#write to xls
#https://xlsxwriter.readthedocs.io/working_with_pandas.html
#https://xlsxwriter.readthedocs.io/example_pandas_column_formats.html
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Dec 2019.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
wikis.to_excel(writer, sheet_name='Dec 2019', float_format = "%0.1f", index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Dec 2019']

# create desired xlsxwriter formats
headers = workbook.add_format({'bold': True})
alignment = workbook.add_format({'align': 'left'})

# apply formats to header and index
worksheet.set_row(0, None, headers)
worksheet.set_column(0,0, 18, alignment)
#set_column(first_col, last_col, width, cell_format, options)

# Add some cell formats
percent = workbook.add_format({'num_format': '0%'})
comma = workbook.add_format({'num_format': '#,##0.0'})


# Set the format on the percent columns.
# Set the format but not the column width.
worksheet.set_column('D:F', 18, percent)
worksheet.set_column('K:K', 18, percent)
worksheet.set_column('M:M', 18, percent)
worksheet.set_column('O:S', 18, percent)

worksheet.set_column('C:C', 18, comma)
worksheet.set_column('G:I', 18, comma)
worksheet.set_column('L:L', 18, comma)
worksheet.set_column('N:N', 18, comma)
worksheet.set_column('U:W', 18, comma)


# Add a sample alternative link format.
blue_format = workbook.add_format({
    'font_color': 'blue',
    'bold':       1,
    'underline':  1,
    'font_size':  12,
})

#Set the format and the width
worksheet.set_column('A:A', 50, blue_format)

worksheet.set_column('B:AF', 18)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
wikis.to_csv("Dec_2019.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
wikis.head()